# Game search - dokumentacja

## Konfiguracja solr

Stworzenie kolekcji.

In [ ]:
solr-8.11.1/bin/solr create -c games -s 2 -rf 2

Indeksowanie danych z grami ze steama.

In [ ]:
solr-8.11.1/bin/post -c games mydata/steam_games.csv

Po uruchomieniu *Solr*, oraz zaindeksowaniu danych należy edytować plik `solr-8.11.1/server/solr-webapp/webapp/WEB-INF/web.xml`.

In [ ]:
  <filter>
    <filter-name>cross-origin</filter-name>
    <filter-class>org.eclipse.jetty.servlets.CrossOriginFilter</filter-class>
    <init-param>
         <param-name>allowedOrigins</param-name>
         <param-value>*</param-value>
    </init-param>
     <init-param>
         <param-name>allowedMethods</param-name>
         <param-value>GET,POST,DELETE,PUT,HEAD,OPTIONS</param-value>
     </init-param>
     <init-param>
         <param-name>allowedHeaders</param-name>
         <param-value>origin, content-type, cache-control, accept, options, authorization, x-requested-with</param-value>
     </init-param>
    <init-param>
        <param-name>supportsCredentials</param-name>
        <param-value>true</param-value>
    </init-param>
    <init-param>
      <param-name>chainPreflight</param-name>
      <param-value>false</param-value>
    </init-param>
  </filter>

  <filter-mapping>
    <filter-name>cross-origin</filter-name>
    <url-pattern>/*</url-pattern>
  </filter-mapping>

Powyższe ustawienia należy dodać przed obecnym `<filter>`, dzięki temu *request* z każdego adresu będzie akceptowany przez *CORS Policy*.

## Frontend

### App

In [ ]:
import React from 'react';
import {ThemeProvider} from 'styled-components';
import theme from '../utils/theme';
import Article from '../components/Article';
import Media from "react-media";
import mobileRender from './mobileRender';
import desktopRender from './desktopRender';
import {queryAction, queryActionDesktop, setQuery} from './queryMethods';

class App extends React.Component {
  state = {
    searchLabel: {
      display: 'block',
      opacity: '1'
    },
    documentIcon: {
      animation: false,
      display: 'block'
    },
    desktopView: {
      aligmentY: 'center',
      logoDisplay: 'flex',
      miniLogoDisplay: 'none',
      searchMaxWidth: '90vw'
    },
    documentIconDesktop: {
      animation: false,
      display: 'none'
    },
    queryInput: '',
    query: '',
    results: [],
    api: 'http://localhost:8983/solr/games/',
    pageNr: 1,
    alphabetSort: false
  }

  queryAction = (e) => {
    queryAction(this, e);
  }

  queryActionDesktop = (e) => {
    queryActionDesktop(this, e);
  }

  setQuery = (e) => {
    setQuery(this, e);
  }

  setAlphabetSort = (e) => {
    this.setState({
      alphabetSort: e.target.checked
    });
  }

  resultsRender(prefix) {
    let results = [];
    let n = (this.state.pageNr - 1) * 10;
    let resultsAlphabetSorted = this.state.results.slice();
    resultsAlphabetSorted.sort((a, b) => a.name[0] > b.name[0] ? 1 : -1);
    if(this.state.alphabetSort)
      results = resultsAlphabetSorted;
    else results = this.state.results;
    return (
      results.slice(n, (n + 10)).map((result, i) =>
        <Article key={`${prefix} ${i}`} name={result.name[0]} tags={result.popular_tags[0]}
                 describe={result.game_description[0]} 
                 date={result.release_date[0]} 
                 query={this.state.query} />)
    );
  }

  mobileRender = () => {
    return mobileRender(this);
  }

  desktopRender = () => {
    return desktopRender(this);
  }

  increasePageNr = () => {
    let resultsLen = this.state.results.length;
    let pageCount = 0;
    if(resultsLen % 10 == 0)
        pageCount = Math.floor(resultsLen / 10);
    else pageCount = Math.floor(resultsLen / 10) + 1;

    if(this.state.pageNr < pageCount) {
        let newPageNr = this.state.pageNr + 1;
        this.setState({
            pageNr: newPageNr
        });
    }
  }

  decreasePageNr = () => {
      if(this.state.pageNr > 1) {
          let newPageNr = this.state.pageNr - 1;
          this.setState({
              pageNr: newPageNr
          });
      }
  }

  render() {
    return (
      <ThemeProvider theme={theme}>
        <Media query={theme.mobile}>
          {this.mobileRender()}
        </Media>
        <Media query={theme.desktop}>
          {this.desktopRender()}
        </Media>
      </ThemeProvider>
    );
  }
}

export default App;

W komponencie *App.js* znajduje się główna logika *frontendu*. 
Stany `searchLabel`, `documentIcon`, `desktopView` oraz `documentIconDesktop` odpowiadają za animację i wyświetlanie elementów przed oraz po wyszukaniem frazy.  
Metody `queryAction`, `queryActionDesktop` oraz `setQuery` zawierają logikę wyszukiwania fraz.  
Metoda `setAlphabetSort` przestawia stan odpowiedzialny za sortowanie.  
Metoda `resultsRender` odpowiada za wyświetlania wyników wyszukiwań.  
Metody `mobileRender` i `desktopRender` odpowiadają odpowiednio za renderowanie po stronie mobilnej rozdzielczości oraz rozdzielczości desktop.
Metody `increasePageNr` oraz `decreasePageNr` odpowiadają za przewijanie stron wyników wyszukiwań.

### queryMethods

In [ ]:
const queryRequest = (component) => {
    fetch(`${component.state.api}select?indent=true&q.op=OR&q=name%3A
    "${component.state.query}"%20popular_tags%3A
    "${component.state.query}"%20game_description%3A"${component.state.query}"&rows=100`)
    .then(response => response.json())
    .then(data => {
      component.setState({results: data.response.docs}); 
      console.log(data.response.docs)
    });
  }
  
  const queryAction = (component, e) => {
      e.preventDefault();
      component.setState({
        query: component.state.queryInput
      });
      setTimeout(() => {
        component.setState({
          searchLabel: {
            display: 'block',
            opacity: '0'
          },
          documentIcon: {
            animation: true,
            display: 'block'
          }
        });
      }, 300);
  
      setTimeout(() => {
        queryRequest(component);
        component.setState({
          searchLabel: {
            display: 'none',
            opacity: '0'
          },
          documentIcon: {
            animation: false,
            display: 'none'
          }
        });
      }, 3000);
  }
  
  const queryActionDesktop = (component, e) => {
      e.preventDefault();
      component.setState({
        query: component.state.queryInput
      });
      setTimeout(() => {
        queryRequest(component);
        component.setState({
          desktopView: {
            alignmentY: 'flex-start',
            logoDisplay: 'none',
            miniLogoDisplay: 'flex',
            searchMaxWidth: '70vw',
          },
          documentIconDesktop: {
            animation: true,
            display: 'block'
          }
        });
      }, 300);
  
      setTimeout(() => {
        component.setState({
          documentIconDesktop: {
            animation: false,
            display: 'none'
          }
        });
      }, 3000);
  }
  
  const setQuery = (component, e) => {
    let value = e.target.value;
    component.setState({
      queryInput: value
    });
  }
  
  export {queryAction, queryActionDesktop, setQuery};

W `queryRequest` znajduje się *request* wysyłany do *solr API*.
Implementacja `queryAction` oraz `queryActionDesktop` ustawia odpowiednio stan `query`, wywołuję metodę `queryRequest` oraz przestawia odpowiednio stany do animacji elementów.
Metoda `setQuery` ustawia stan `queryInput`, który zmienia się na bieżąco gdy wpisujemy coś w *search input*, a stan `query` aktualizuje się dopiero gdy potwierdzimy zapytanie.